# Analyse reserve

Analyse the reseve position for a strategy.

 - A script for detailed calculations of a resever position


In [1]:
trade_executors = [
    "https://enzyme-polygon-eth-usdc-sls.tradingstrategy.ai",
]



In [2]:
import requests
from tradeexecutor.state.state import State

total_downloaded = 0

def download_state(url) -> State:
    global total_downloaded
    resp = requests.get(f"{url}/state")

    if len(resp.content) == 0:
        raise RuntimeError(f"Could not download: {url}")

    try:
        state = State.read_json_blob(resp.text)
    except Exception as e:
        raise RuntimeError(f"Could not decode: {url}") from e
    total_downloaded += len(resp.content)
    return state

states = [download_state(url)for url in trade_executors]

print(f"Downloaded {total_downloaded:,} bytes")

Downloaded 142,880 bytes


## Denormalised balance

In [3]:

state: State
for state in states:
    portfolio = state.portfolio
    asset, exchange_rate = state.portfolio.get_default_reserve_asset()
    position = portfolio.get_default_reserve_position()
    print(f"Reserves: {asset} at {exchange_rate} {asset.token_symbol} / USD")
    print(f"Amount: {position.get_quantity()} {asset.token_symbol}")
    print(f"Equity: {position.get_total_equity()} USD")
    print(f"Value: {position.get_value()} USD")
    print(f"Balance updates: {position.get_balance_update_quantity()} {asset.token_symbol}")

Reserves: <USDC at 0x2791bca1f2de4661ed88a30c99a7a9449aa84174> at 1.0 USDC / USD
Amount: 17.27981900000000000000000000 USDC


## Individual events

For the last analysed executor.

In [4]:
from tradeexecutor.analysis.position import display_reserve_position_events

df = display_reserve_position_events(position)
display(df)


,Cause,At,Quantity,Dollar value,Address
1,deposit,<BalanceUpdate #1 deposit 8.676756 for strateg...,8.676756,8.676756,
